<a href="https://colab.research.google.com/github/PingPingE/Learn_ML_DL/blob/main/Practice/Hands_On_ML/ch16-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_gpu
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import pandas as pd

# RNN과 어텐션을 사용한 자연어 처리
- 자연어 문제를 위해 많이 사용하는 방법은 순환 신경망임
- 문자 단위 RNN: 문장에서 다음 글자를 예측
  - 상태가 없는 RNN: 각 반복에서 무작위하게 택한 텍스트의 일부분으로 학습하고 나머지 텍스트에서 어떤 정보도 사용하지 않음 
  - 상태가 있는 RNN: 훈련 반복 사이에 은닉 상태를 유지하고 중지된 곳에서 이어서 상태를 반영 -> 긴 패턴 학습 가능

- 단어 단위(시퀀스) RNN
- RNN 기반의 인코더-디코더 구조
- 어텐션 메커니즘
  - 각 타임 스텝에서 모델이 집중해야 할 입력 부분을 선택하도록 학습하는 신경망 구성 요소



## Char-RNN을 사용해 셰익스피어 문체 생성하기
- Char-RNN을 사용해서 한 번에 한 글자씩 새로운 텍스트를 생성할 수 있다.

### 훈련 데이터셋 만들기

In [2]:
url="https://homl.info/shakespeare"
filepath=keras.utils.get_file("shakespeare.txt", url)
with open(filepath) as f:
  text=f.read()

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
len(text)#=== 글자 수가 백만 개 이상

1115394

In [4]:
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

#### 모든 글자를 정수로 인코딩하기 -> 케라스의 Tokenizer클래스 아용

In [5]:
tokenizer=keras.preprocessing.text.Tokenizer(char_level=True)#=== 단어 수준이 아닌 글자 수준 인코딩
tokenizer.fit_on_texts(text)#=== 텍스트에 훈련

------
텍스트에 사용되는 모든 글자를 찾아 글자 ID에 매핑(ID는 1부터 시작)
<br><br>
- 간단한 테스트

In [6]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [7]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

- 전체 텍스트를 인코딩하여 각 글자를 ID로 나타내기
  - 1부터 시작하므로 -1해서 0부터 시작하는 걸로 바꿔보기

In [8]:
max_id =len(tokenizer.word_index)
max_id  #===== 즉, 고유 글자가 39개

39

In [9]:
[encoded] = np.array(tokenizer.texts_to_sequences([text]))-1

In [10]:
encoded

array([19,  5,  8, ..., 20, 26, 10])

### 순차 데이터셋을 나누는 방법
- 훈련/검증/테스트 세트가 <strong>중복되지 않도록</strong> 만드는 것이 매우 중요
  - 예를 들어 텍스트 처음 90%/5%/5%

- 시계열을 다룰 때는 보통 <strong>시간</strong>에 따라 나눈다.(안전하다)
  - ex)100개의 회사의 재정 건전성 관련 데이터 다루기 
    - 많은 회사들이 <strong>강하게 상호 연관</strong>되어 있을 가능성이 높다.
    - 훈련 세트와 테스트 세트에 상호 연관된 회사가 있다면 <strong>테스트 세트에서 측정한 일반 오차가 유용하지 않을 것임</strong>
    - => 따라서 회사를 기준으로 나누는 등 다른 방법 보다 시간에 따라 나누는 것이 일반적이고, 안전함

- 시계열이 <strong>충분히 안정적</strong>인지 꼭 확인해야한다.
  - 일반적으로는, 시계열의 패턴이 '변하지 않는다'고 가정하지만, 일부는 그렇지 않음
    - ex) 금융 시장은 변덕스럽다. 트레이더가 패턴을 발견한 뒤 적용하려 하면 사라짐
  - 안정성 확인 방법: <strong>시간에 따라 검증 세트에 대한 모델의 오차 그려보기</strong>
    - 모델이 검증 세트 <strong>마지막보다 첫 부분에서 성능이 더 좋다면</strong> 이 시계열이 충분히 안정되지 않은 것일 수 있음
      - 이 경우, <strong>더 짧은 시간 간격</strong>으로 모델을 훈련하는 것이 좋다.

#### train: 텍스트의 처음 90%

In [11]:
dataset_size=tokenizer.document_count#== 전체 글자 개수
train_size=dataset_size*90//100 #=== 텍스트의 처음 90%
dataset= tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset

<TensorSliceDataset shapes: (), types: tf.int64>

### 순차 데이터를 윈도 여러 개로 자르기
- 해당 text는 백만 개 이상의 글자로 이루어진 시퀀스 하나
- 즉, 이대로 RNN으로 훈련 시키면, 백만 개의 층이 있는 심층 신경망과 같게 된다.
- 따라서 <strong>window()메서드</strong>를 사용해서 긴 문자열을 짧은 부분 문자열로 변환한다.
- 이 짧은 부분 문자열만큼만 역전파하는게 <strong>TBPTT(Truncated Backpropagation Through Time)</strong>
- BPTT vs TBPTT

  <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FdawXWw%2FbtqD2dngo5l%2FrvitZfPrq5bXlHPUfy07J0%2Fimg.png" width=50% />
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbiRY38%2FbtqD2bweMfV%2FXiaxKlF44miJP4IHATTfQk%2Fimg.png" width=50%/>

In [12]:
n_steps=100
window_length=n_steps+1
dataset=dataset.window(window_length,shift=1,drop_remainder=True) 

--------------
- window_length: 한 번에 얼마나 볼것인지(너무 짧게 만들면 긴 패턴을 학습할 수 없으므로 적절하게 조절해야함)
- shift: 윈도우를 얼마만큼 움직일 것인지(기본값=window 길이 )
- drop_remainder:  True로 하면 모든 윈도우가 동일한 개수의 글자를 포함하도록한다.
<br><br>

In [13]:
dataset= dataset.flat_map(lambda window: window.batch(window_length)) #=== 윈도우마다 batch(window_length)호출

--------
- flat_map()메서드를 통해서 데이터셋을 변환하고 평평하게 만든다
- ex) 기존 데이터셋(ds): {{1,2},{3,4,5,6}} <br>
flat_map(lambda ds: ds.batch(2)) ->   {[1,2],[3,4],[5,6]} : 텐서 2개를 가진 데이터셋
<br><br>

- 섞고, 타깃 값(마지막 1개 글자) 분리

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FY65A7%2FbtqEWB10RrH%2Ff1Ngbw1V4OTxuhbpKRlkr1%2Fimg.png"/>

- 위 예시는 윈도 크기가 11 , 배치 크기가 3인 경우이다.
- input이 [2,12] 구간이라면, target은 [3,13]구간의 문자가 되겠다.
  - 즉, [2,12]구간의 문자들로 학습을 하면 13번째 문자를 추가로 맞춰야 하는 것

In [14]:
batch_size=32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset=dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

- 입력 특성 원-핫 벡터
  - 고유한 글자 수가 적기 때문(39개)

In [15]:
dataset= dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
)

In [16]:
dataset = dataset.prefetch(1)

### Char-RNN 모델 만들고 훈련하기
- 이전 글자 100개를 기반으로 다음 글자를 예측하기 위해 
  - 유닛 128개를 가진 GRU층 2개
  - 입력과 은닉 상태에 20% 드롭아웃(dropout, recurrent_dropout)
  - 출력층은 각 글자에 대한 확률을 출력할 것이므로 softmax, 유닛 max_id개

In [17]:
model= keras.models.Sequential([
                                keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2, recurrent_dropout=0.2),
                                keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
                                keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, None, 128)         64896     
_________________________________________________________________
gru_1 (GRU)                  (None, None, 128)         99072     
_________________________________________________________________
time_distributed (TimeDistri (None, None, 39)          5031      
Total params: 168,999
Trainable params: 168,999
Non-trainable params: 0
_________________________________________________________________


In [22]:
cd train

/content/drive/My Drive/Colab Notebooks/HandsOn/train


In [23]:
#==학습이 오래 걸리니 중간 중간 저장하자
#check_point = keras.callbacks.ModelCheckpoint('rnn_{epoch:02d}.h5', monitor='loss', save_freq=3)
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam')
history=model.fit(dataset, epochs=20, callbacks=[check_point]) #====너어어어ㅓ무 오래 걸려서 나중에...

In [25]:
def preprocess(texts):
  X=np.array(tokenizer.texts_to_sequences(texts)) -1
  return tf.one_hot(X,max_id)

In [ ]:
X_new=preprocess([["How are yo"], ["I'm fin"]])
y_pred=model.predict_classes(X_new)
tokenizer.sequences_to_texts(y_pred+1)[0][-1] #첫 번째 문장을 봤을 때 이어질 다음 문자는? -> 학습 20 에포크동안 하면 'u' 잘 나온다고 함

### 가짜 셰익스피어 텍스트 생성하기
Char-RNN모델로 새로운 텍스트를 생성하려면?
- Char-RNN 모델이 예측한 다음 글자를 텍스트 끝에 추가 -> input -> 추가 -> input .... 
  - 근데 이 방식으로 하면 같은 단어가 계속 반복 됨
<br>
- tf.random.categorical()함수를 사용해서 모델이 추정한 확률을 기반으로 다음 글자를 무작위로 선택한다면 더 다채로운 텍스트 생성 가능
  - 다양성 제어 파라미터: temperature
    - 예측 확률에 로그를 취하고(로짓) temperature로 나눔
    - 0에 가까울수록 높은 확률을 가진 글자가 선택될 가능성이 더 높아져서 다채로움이 사라짐

In [29]:
def next_char(text, temperature=1):
  X_new = preprocess([text])
  y_proba= model.predict(X_new)[0,-1:,:]
  rescaled_logits=tf.math.log(y_proba)/temperature #==== temperature로 나누는 부분
  char_id=tf.random.categorical(rescaled_logits, num_samples=1)+1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

- next_char()함수를 반복 호출해서 다음 글자를 얻고, 텍스트에 추가

In [28]:
def complete_text(text, n_chars=50, temperature=1):
  for _ in range(n_chars):
    text += next_char(text, temperature)
  return text

----------
- 해당 모델은 1에 가까운 온도에서 가장 잘 작동한다고 함
- 해당 모델은 n_steps=100을 넘어가면 훈련이 더 어려워질 것임
  - LSTM, GRU셀도 매우 긴 시퀀스는 다루기 어렵다<br>
=> 상태가 있는 RNN을 살펴보자

### 상태가 있는 RNN
